In [1]:
from langchain_openai import AzureChatOpenAI
from langchain.prompts import ChatPromptTemplate
import os

import truststore
truststore.inject_into_ssl()
import numpy as np
import pandas as pd
from tqdm import tqdm
import time
import seaborn as sns
import numpy as np
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
os.environ["APP_CLIENT_ID"] = "zizhang-chen-research-app"
os.environ["APP_CLIENT_SECRET"] = "cQPcueFP7tDrimbf8NW2GAHcHeQa"
from llm_idam_token_generator.idam_token_generator import get_idam_token
from langchain_community.callbacks import get_openai_callback
from langchain_openai import OpenAI
import tiktoken

In [2]:
end_point_list = [
    'https://lmaas-beta.ai.gehealthcare.com',
    "https://openai-llm-frontdoor-hma7evbthrd4cugn.a01.azurefd.net"
]

model_list = ["gpt-35-turbo-16k", "gpt-4-32k-beta", "gpt-4o"]

# APIM_KEY = "8b96051ed6b84e4dad762fdc9f8c809e"

OPENAI_ENDPOINT = end_point_list[0]
OPENAI_DEPLOYMENT_MODEL = model_list[0]
OPENAI_AZURE_API_VERSION = "2023-12-01-preview"
# OPENAI_AZURE_API_VERSION = '2024-02-01'
OPENAI_TYPE = "azure"

In [3]:
# help(AzureChatOpenAI)

llm = AzureChatOpenAI(
    api_key="xxx",  # This is not playing any role, but required as per OpenAI sdk. So any random could be passed.
    azure_endpoint=OPENAI_ENDPOINT,
    deployment_name=OPENAI_DEPLOYMENT_MODEL,
    openai_api_version=OPENAI_AZURE_API_VERSION,
    n = 2,
    default_headers={
        'Authorization': f'Bearer {get_idam_token()}',
        'Content-Type': 'application/json'
    }
)

PID:15007 INFO llm_idam_token_generator.idam_token_generator - Client ID: zizhang-chen-research-app - Generating new token.
PID:15007 INFO llm_idam_token_generator.idam_token_generator - Client ID: zizhang-chen-research-app - All required environment variables are present.
/opt/anaconda3/lib/python3.11/site-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
PID:15007 INFO llm_idam_token_generator.idam_token_generator - Client ID: zizhang-chen-research-app - IDAM Access Token is generated
/opt/anaconda3/lib/python3.11/site-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-w

In [4]:
def generate_prompt_task(path):
    with open(path) as f:
        prompt = f.readlines()
    return prompt

def split_string_into_two_parts(text):
    lines = text.split('\n')
    first_part = lines[:2]
    second_part = lines[2:]

    first_part_string = '\n'.join(first_part)
    second_part_string = '\n'.join(second_part)
    
    return first_part_string, second_part_string


def combin_prompt_no_cot(general_prompt, person_info, medical_event):
    template_by_task = '**Task:**\n' + general_prompt + '\n'

    template_info = '**Patient age and demographic information:**\n' + person_info + '\n'

    template_events = '**Medical Events:**\n' + medical_event + '\n'
    
    template_events_answer_style = "**Answer requirement:**\nPlease only answer with 'Yes' or 'No'"

    return template_by_task + '\n' + '{events}' , template_info + '\n' + template_events + '\n' + template_events_answer_style

def calculate_row_entropy(df):
    def entropy(row):
        counts = row.value_counts(normalize=True)
        return -np.sum(counts * np.log2(counts + np.finfo(float).eps)) 

    return df.apply(entropy, axis=1)

def num_tokens_from_string(string, encoding_model = 'gpt-3.5-turbo') -> int:
    encoding = tiktoken.encoding_for_model(encoding_model)
    num_tokens = len(encoding.encode(string))
    return num_tokens

def generate_prompt_task(lab_test, lab_test_prediction):

    task = f"You are an experienced doctor. Based on the provided patient age, demographic information and medical events, use your medical knowledge and reasoning to: {lab_test_prediction} right after the given medical events.\nPlease only answer with 'Yes' or 'No'. Yes if the patient lab results {lab_test} come back as normal, No if the patient lab results remain moderate or high. Please respond 'Yes' if it is at all plausible, only use 'No' if absolutely certain otherwise."
    return task

In [5]:
df = pd.read_csv('data/labtest_patient_description.csv')
description_list = df['description'].values
lab_test_list = [i.split('_')[1] for i in df.columns[2:7]]
lab_test_prediction = [
    'Predict whether an anemia lab comes back as normal (>=120 g/L), mild (>=110 and <120 g/L), moderate (>=70 and <110 g/L), or severe (<70 g/L),',
    'Predict whether a hypoglycemia lab comes back as normal (>=3.9 mmol/L), mild (>=3.5 and <3.9 mmol/L), moderate (>=3 and <3.5 mmol/L), or severe (<3 mmol/L),',
    'Predict whether a hyponatremia lab comes back as normal (>=135 mmol/L), mild (>=130 and <135 mmol/L), moderate (>=125 and <130 mmol/L), or severe (<125 mmol/L),',
    'Predict whether a thrombocytopenia lab comes back as normal (>=150 10^9/L), mild (>=100 and <150 10^9/L), moderate (>=50 and <100 10^9/L), or severe (<50 10^9/L),',
    'Predict whether a hyperkalemia lab comes back as normal (<=5.5 mmol/L), mild (>5.5 and <=6mmol/L), moderate (>6 and <=7 mmol/L), or severe (>7 mmol/L),'
]

In [6]:
lab_test_list

['anemia', 'hypoglycemia', 'hyponatremia', 'thrombocytopenia', 'hyperkalemia']

In [7]:
# for idx in [0]:
for idx in range(5):
    # for idx in [4]:
    # out_csv_names = lab_test_list[idx].split('/')[-1].replace('.txt', '.csv')
    out_csv_names = lab_test_list[idx] + '.csv'
    general_prompt = generate_prompt_task(
        lab_test=lab_test_list[idx],
        lab_test_prediction=lab_test_prediction[idx])
    gt_list = df['value_' + lab_test_list[idx]].values

    with get_openai_callback() as cb:

        answer_list_all = []

        for i in tqdm(range(len(df))):
            # for i in tqdm(range(1)):
            answer_list_5 = []
            for _ in range(5):
                # for _ in range(1):
                current_description = description_list[i]
                person_info, medical_events = split_string_into_two_parts(
                    current_description)
                c, e = combin_prompt_no_cot(general_prompt, person_info,
                                            medical_events)

                prompt = ChatPromptTemplate.from_template(c)
                chain = prompt | llm
                result = chain.invoke({'events': e})
                answer_list_5.append(result.content)
            answer_list_all.append(answer_list_5)

        df_pred = pd.DataFrame(answer_list_all)
        df_pred.columns = ['pred_1', 'pred_2', 'pred_3', 'pred_4', 'pred_5']
        df_pred['gt'] = list(gt_list)
        # df_pred.to_csv(
        #     os.path.join('results/gpt35/baseline_labtest/',
        #                  out_csv_names + '.csv'))
        # df_pred.to_csv(
        #     os.path.join('results/gpt4/baseline_labtest_v2/',
        #                  out_csv_names + '.csv'))
        df_pred.to_csv(os.path.join('results/gpt35/baseline_labtest_v2/', out_csv_names + '.csv'))

  0%|          | 0/100 [00:00<?, ?it/s]PID:15007 INFO httpx - Client ID: zizhang-chen-research-app - HTTP Request: POST https://lmaas-beta.ai.gehealthcare.com/openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-12-01-preview "HTTP/1.1 200 OK"
PID:15007 INFO httpx - Client ID: zizhang-chen-research-app - HTTP Request: POST https://lmaas-beta.ai.gehealthcare.com/openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-12-01-preview "HTTP/1.1 200 OK"
PID:15007 INFO httpx - Client ID: zizhang-chen-research-app - HTTP Request: POST https://lmaas-beta.ai.gehealthcare.com/openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-12-01-preview "HTTP/1.1 200 OK"
PID:15007 INFO httpx - Client ID: zizhang-chen-research-app - HTTP Request: POST https://lmaas-beta.ai.gehealthcare.com/openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-12-01-preview "HTTP/1.1 200 OK"
PID:15007 INFO httpx - Client ID: zizhang-chen-research-app - HTTP Reques

In [8]:
out_csv_names

'hyperkalemia.csv'

In [9]:
cb

Tokens Used: 2237265
	Prompt Tokens: 2236265
	Completion Tokens: 1000
Successful Requests: 500
Total Cost (USD): $6.712795000000014

In [10]:
print(e)

**Patient age and demographic information:**
The patient was 74 years old at the prediction time.
The patient has the following demographic information: White,FEMALE,Hispanic or Latino.

**Medical Events:**
At August 17, 2019:
1 events Platelets [#/volume] in Blood by Automated count recorded.with values: 282.0. 
1 events Eosinophils [#/volume] in Blood by Automated count recorded.with values: 0.1899999976158142. 
1 events Monocytes [#/volume] in Blood by Automated count recorded.with values: 0.8500000238418579. 
1 events Lymphocytes/100 leukocytes in Blood by Automated count recorded.with values: 16.0. 
1 events Immature granulocytes [#/volume] in Blood by Automated count recorded.with values: 0.1899999976158142. 
1 events Anion gap in Serum or Plasma recorded.with values: 9.0. 
1 events Nucleated erythrocytes/100 leukocytes [Ratio] in Body fluid recorded.with values: 0.20000000298023224. 
1 events Immature granulocytes/100 leukocytes in Blood by Automated count recorded.with values: 

In [11]:
answer_list_5

['No', 'No', 'Yes', 'No', 'Yes']

In [12]:
answer_list_all, cb, len(answer_list_all), 24

([['No', 'No', 'No', 'No', 'No'],
  ['No', 'No', 'No', 'No', 'No'],
  ['No', 'Yes', 'No', 'No', 'No'],
  ['No', 'No', 'No', 'No', 'No'],
  ['No', 'No', 'No', 'No', 'No'],
  ['No', 'No', 'No', 'No', 'No'],
  ['Yes', 'No', 'No', 'No', 'No'],
  ['No', 'No', 'No', 'No', 'No'],
  ['No', 'No', 'No', 'No', 'No'],
  ['Yes', 'No', 'No', 'No', 'No'],
  ['Yes', 'No', 'No', 'No', 'No'],
  ['No', 'No', 'No', 'No', 'No'],
  ['No', 'No', 'No', 'No', 'No'],
  ['No', 'Yes', 'No', 'No', 'No'],
  ['No', 'Yes', 'Yes', 'No', 'Yes'],
  ['Yes', 'No', 'No', 'No', 'No'],
  ['No', 'No', 'No', 'No', 'Yes'],
  ['No', 'No', 'No', 'No', 'No'],
  ['No', 'No', 'No', 'No', 'No'],
  ['No', 'No', 'No', 'No', 'No'],
  ['No', 'No', 'No', 'No', 'Yes'],
  ['No', 'Yes', 'No', 'No', 'No'],
  ['No', 'No', 'No', 'No', 'No'],
  ['No', 'No', 'Yes', 'No', 'No'],
  ['No', 'No', 'No', 'No', 'No'],
  ['Yes', 'No', 'No', 'No', 'No'],
  ['No', 'No', 'No', 'No', 'No'],
  ['No', 'No', 'No', 'No', 'No'],
  ['No', 'No', 'No', 'No', 'No'],
